# LDA match finder

In [17]:
import pandas as pd
topics=int(input("enter topic : "))
bay=pd.read_csv(f"../nb_outputs/news_20/baysian_topic_{topics}.csv")
bay.drop(["Unnamed: 0"],axis=1,inplace=True)

lda=pd.read_csv(f"../LDA_outputs/news_20/{topics}_topics.csv")
lda.drop(["Unnamed: 0"],axis=1,inplace=True)


In [18]:
lda

,topic_0,topic_1,topic_2,topic_3,topic_4,index
0,0.005418,0.005288,0.005263,0.978640,0.005391,0
1,0.769249,0.004018,0.004002,0.004118,0.218612,1
2,0.473737,0.001330,0.001325,0.436159,0.087449,2
3,0.899142,0.025379,0.025000,0.025191,0.025288,3
4,0.710022,0.004809,0.004447,0.004592,0.276131,4
...,...,...,...,...,...,...
18461,0.109688,0.048197,0.002357,0.749589,0.090170,18841
18462,0.429388,0.016797,0.348793,0.017134,0.187888,18842
18463,0.004396,0.004277,0.004255,0.334195,0.652877,18843
18464,0.631153,0.361633,0.002382,0.002416,0.002416,18844


In [19]:
import pickle
with open(f"../nb_outputs/news_20/missing_values_{topics}.pkl","rb") as f:
    missed_idx=pickle.load(f) #got this from nb_gnn 
# missed_idx #drop these values from plsa  #these are the index which are not in the baysian_topic_5 dataset 

In [20]:
# len(news_20)-len(missed_idx),len(bay)

In [21]:
news_20=pd.read_csv("../datasets/20news.csv")
# print(news_20.shape,bay.shape[0]+len(missed_idx))
#now using this information generate new indexes
#from the original subtract(eliminate) the missing idx and the new idx 
original_idx=list(range(news_20.shape[0]))
# print(len(original_idx))
for item in missed_idx:
    original_idx.remove(item)
# len(original_idx),bay.shape

In [22]:
# make a new column in bayes and append these indexes
bay["index"]=original_idx

# Now remove the rows which was not common to both the datasets

In [23]:
bay_idx,lda_idx=set(bay["index"]),set(lda["index"])

In [24]:
idx_to_keep=bay_idx.intersection(lda_idx)

In [25]:
len(idx_to_keep)

10743

In [26]:
cleaned_bay=bay[bay["index"].isin(idx_to_keep)]
cleaned_lda=lda[lda["index"].isin(idx_to_keep)]


In [27]:
temp=cleaned_lda["index"].to_list()
for i in cleaned_bay["index"].to_list():
    if i not in temp:
        print(i,"what the fuck")
    # break

In [28]:
cleaned_bay.shape,cleaned_lda.shape

((10743, 6), (10743, 6))

In [29]:
cleaned_bay.index=cleaned_bay["index"]
cleaned_lda.index=cleaned_lda["index"]

In [30]:
cleaned_lda.drop(["index"],axis=1,inplace=True)
cleaned_bay.drop(["index"],axis=1,inplace=True)

C:\Users\rbw19\AppData\Local\Temp\ipykernel_20844\986028581.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_lda.drop(["index"],axis=1,inplace=True)


In [31]:
cleaned_lda.columns=[str(i) for i in range(cleaned_lda.shape[1])]

In [32]:
cleaned_lda

,0,1,2,3,4
index,,,,,
2,0.473737,0.001330,0.001325,0.436159,0.087449
3,0.899142,0.025379,0.025000,0.025191,0.025288
4,0.710022,0.004809,0.004447,0.004592,0.276131
5,0.006454,0.006292,0.006250,0.052006,0.928997
7,0.994655,0.001331,0.001329,0.001340,0.001344
...,...,...,...,...,...
18840,0.001802,0.001785,0.001787,0.992839,0.001787
18841,0.109688,0.048197,0.002357,0.749589,0.090170
18842,0.429388,0.016797,0.348793,0.017134,0.187888


# Finding match accuracy without order

In [33]:
import pandas as pd

def accuracy(n_largest,match_threshold,no_topics): #instead of gntm here we have PLSA or other probabilitic models

    bay=cleaned_bay
    gnt=cleaned_lda #changing the gntm to plsa
    print("read sucessfully")
    # bay.drop(["Unnamed: 0"],axis=1,inplace=True) 
    # gnt.drop(["Unnamed: 0"],axis=1,inplace=True) 
    bay,gnt=bay.transpose(),gnt.transpose()
    bay_top_3,gnt_top_3=[],[]

    for col in range(bay.shape[1]):
        try:
            bay_top_3.append(bay.nlargest(n_largest,col).index) 
        except:
            pass

    for col in range(gnt.shape[1]):
        try:
            gnt_top_3.append(gnt.nlargest(n_largest,col).index)
        except:
            pass

    bay_col_names=[]
    gnt_col_names=[]
    for i in range(1,n_largest+1):
        bay_col_names.append(f'bay_top{str(i)}')
        gnt_col_names.append(f'gnt_top{str(i)}')
        
    bay_top3_df=pd.DataFrame(bay_top_3,columns=bay_col_names)
    gnt_top3_df=pd.DataFrame(gnt_top_3,columns=gnt_col_names)
    final_matches=[]

    for gnt_out,bay_out in zip(gnt_top_3,bay_top_3):
        matches=[]
        for out in gnt_out:
            if out in bay_out:
                matches.append(True)
            else:
                matches.append(False)
        percentage_of_acc=(matches.count(True)/len(matches))*100

        if percentage_of_acc >= match_threshold: 
            final_matches.append(True)
        else:
            final_matches.append(False)
            
    result=pd.concat([bay_top3_df,gnt_top3_df,pd.DataFrame(final_matches,columns=["match"])],axis=1)
    # decision=str(input("Do you want to save the o/p(yes/no)?"))
    decision="yes"
    if decision=="Yes" or decision=="yes" or decision=="YES":
        # file_name=str(input("enter file name :"))
        file_name=f"thres_{match_threshold}"
        result.to_csv(f"{file_name}.csv")
    
    try:
        result["match"].value_counts()[True]
    except:
        print('accuracy : 0 % ')
        with open(f"{no_topics}_lda.txt","a") as op:
            op.writelines(f"\n Number of topics : {no_topics} \n N-largest : {n_largest} \n Match Threshold :{match_threshold} \n accuracy : 0")
        
    try:
        result["match"].value_counts()[False]
    except:
        print('accuracy : 100%')
        with open(f"{no_topics}_lda.txt","a") as op:
            op.writelines(f"\n Number of topics : {no_topics} \n N-largest : {n_largest} \n Match Threshold :{match_threshold} \n accuracy : 100")

    try:
        accuracy=result["match"].value_counts()[True]/(result["match"].value_counts()[True]+result["match"].value_counts()[False])
        print("The accuracy is :",accuracy)
        with open(f"{no_topics}_lda.txt","a") as op:
            op.writelines(f"\n Number of topics : {no_topics} \n N-largest : {n_largest} \n Match Threshold :{match_threshold} \n accuracy : {accuracy}")
    except:
        print("result already shown")


# topics=str(input("enter no. of topics :"))
# n_largest=int(input("enter n_largest : "))

# thres=[60,80,100]

# for threshold in thres:
#     print("topics:",topics)
#     print("n largest :",n_largest)
#     print("Threshold is :",threshold)
#     accuracy(n_largest,threshold,topics)

In [34]:
thres=[60,80,100]
for topic,nlar in zip([5,5,5,5],[2,3,4,5]):
    for i in thres:
        print("topics:",topic)
        print("n largest :",nlar)
        print("Threshold is :",i)
        accuracy(nlar,i,topic)

topics: 5
n largest : 2
Threshold is : 60
read sucessfully
The accuracy is : 0.03756561679790026
topics: 5
n largest : 2
Threshold is : 80
read sucessfully
The accuracy is : 0.03756561679790026
topics: 5
n largest : 2
Threshold is : 100
read sucessfully
The accuracy is : 0.03756561679790026
topics: 5
n largest : 3
Threshold is : 60
read sucessfully
The accuracy is : 0.4187992125984252
topics: 5
n largest : 3
Threshold is : 80
read sucessfully
The accuracy is : 0.03264435695538058
topics: 5
n largest : 3
Threshold is : 100
read sucessfully
The accuracy is : 0.03264435695538058
topics: 5
n largest : 4
Threshold is : 60
read sucessfully
accuracy : 100%
result already shown
topics: 5
n largest : 4
Threshold is : 80
read sucessfully
